# Garden Tutorial Finale: Execute Your Model Remotely

This notebook is part of [the Garden AI tutorial](https://garden-ai.readthedocs.io/en/latest/user_guide/tutorial/).
This notebook will show you how to invoke your iris classification model now that you've published it.

### Install the Garden SDK

In [1]:
# We'll need to install Garden in this Colab session
!pip install garden-ai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.0/56.0 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.8/95.8 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.1/148.1 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.1/299.1 kB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 30.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 257.3/257.3 kB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 53.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━

## Create a GardenClient and Log in with Globus

We need a GardenClient to run entrypoint functions. You'll have to log in like you did on the command line. Run the cell, click through the link, and paste the authorization code in the text box. Then you'll have a working GardenClient.

In [2]:
from garden_ai import GardenClient
garden_client = GardenClient()

Authenticating with Globus in your default web browser:

Please enter the code here :

c93GpT5L5WbmVVKTPkmxFkEu6xXq6w


### Fetch Your Published Garden

Now you will pull down a copy of your garden from the Garden backend. If you don't remember your garden's doi, check with `garden-ai garden list` in a terminal on your computer.

In [3]:
your_garden_doi = '10.26311/agt2-t976' # Paste in your garden's DOI here!
tutorial_garden = garden_client.get_published_garden(your_garden_doi)
tutorial_garden

PublishedGarden(title='Tutorial Garden', authors=['Ryan Jacobs'], contributors=[], doi='10.26311/agt2-t976', description='ASR model test', publisher='Garden-AI', year='2024', language='en', tags=[], version='0.0.1', entrypoints=[RegisteredEntrypoint(doi='10.26311/znn8-gn85', title='ASR regression model', authors=['Ryan Jacobs'], short_name='predict_asr', description='Random forest and Gaussian process regression models to predict the stability and ASR for perovskite oxides useful for oxygen reduction and evolution catalysis', year='2024', tags=['scikit-learn', 'mastml', 'tutorial'], models=[ModelMetadata(model_identifier='https://github.com/uw-cmg/ASR_model', model_repository='GitHub', model_version='main', datasets=[])], repositories=[], papers=[], doi_is_draft=True, func_uuid=UUID('16705f31-4c05-48f9-8f02-d02395d661a2'), container_uuid=UUID('0e314eae-7248-4c70-a467-9b5991218cb4'), base_image_uri='gardenai/base:python-3.10-jupyter-sklearn', full_image_uri='public.ecr.aws/x2v7f8j4/garden-containers-prod:pushed-20240226-142251315131', notebook_url='https://pipeline-notebooks-prod.s3.amazonaws.com/RJACOBS3@WISC.EDU/asr_notebook.ipynb-3557ee5e52a1ea4537d360814120ea5bc7343d1492ef655ce7956f8285c4607f.ipynb', steps=[Step(function_name='predict_asr', function_text="@garden_entrypoint(\n    metadata=entrypoint_meta,  \n    model_connectors=[github_asr_connector], \n    garden_doi=my_garden_doi\n)\ndef predict_asr(comp_list, elec_list):\n\n    import shutil, zipfile\n    # The HFConnector.stage() method downloads the model weights from Hugging Face.\n    # Be sure to only call .stage inside of an entrypoint function!\n    import os\n    if not os.path.exists('ASR_model'):\n        download_path = github_asr_connector.stage()\n        shutil.move('gh_model', 'ASR_model')\n        with zipfile.ZipFile('ASR_model/Stability_model.zip', 'r') as zip_ref:\n            zip_ref.extractall()\n        shutil.move('Stability_model', 'ASR_model')\n\n    import sys\n    sys.path.append('ASR_model')\n\n    from model_predict_df import make_predictions\n    \n    preds = make_predictions(comp_list, elec_list)\n\n    return preds\n", description='Top level entrypoint function')], test_functions=["@entrypoint_test(predict_asr)\ndef test_the_model():\n    example_comps = ['La0.8Sr0.2MnO3', 'Ba0.5Sr0.5Co0.8Fe0.2O3']\n    example_elecs = ['zirconia', 'ceria']\n    result = predict_asr(example_comps, example_elecs)\n    return result\n"])], entrypoint_aliases={})

### Execute Your Model

The entrypoint function from the notebook you used to publish will be a method on the garden it was published to. We just need to pass it the same input we passed it locally and call it. Garden will serialize your input, send it to a worker on the Garden demo computing endpoint, and send the response back to the notebook.

#### Entrypoint Cold Start

Your entrypoint might take a minute or so to run the first time you execute it. That's because the remote computing endpoint has to download the underlying container. But after that your entrypoint will be warmed up and only take a few seconds.

In [5]:
#example_input = [[5.1, 3.5, 1.4, 0.2]]
#tutorial_garden.classify_irises(example_input)
example_comps = ['La0.8Sr0.2MnO3', 'Ba0.5Sr0.5Co0.8Fe0.2O3']
example_elecs = ['zirconia', 'ceria']
result = tutorial_garden.predict_asr(example_comps, example_elecs)
result

Output()

,Compositions,Electrolytes,Cost ($/kg),Stability @ 500C (meV/atom),ASR barrier (eV),log ASR at 500C (Ohm-cm2) (RF),log ASR error (Ohm-cm2) (RF),log ASR at 500C (Ohm-cm2) (GPR),log ASR error (Ohm-cm2) (GPR)
0,La0.8Sr0.2MnO3,zirconia,169.046056,214.202900,1.564999,3.552795,0.575237,3.543993,0.521608
1,Ba0.5Sr0.5Co0.8Fe0.2O3,ceria,8.750681,130.604608,1.205874,0.206754,0.553044,0.317634,0.507966


## Wrapping Up

If you see `['setosa']`, congratulations, you have successfully published and executed your first model with Garden! You can return to [the online tutorial](https://garden-ai.readthedocs.io/en/latest/user_guide/tutorial/) now.